# Playing Nim Game with Reinforcement Learning

We'll introduce three types of players, the *Expert*, *Random*, and the *Qlearner*. The *Expert* is very good at this game, in fact its algorithm is backed by sound mathematics. The *Qlearner* picks its move by reading off a value from a table called the *Q table*, basically picking the maximum value indexed by the three values corresponding to the numbers of items in each of the three piles. This combination of key values represents the *state* in a game. We'll explain more about the Q table and how to update them in the last section when we discuss how to train the *Qlearner* with reinforcement learning.

In [1]:
from random import randint, random, choice
import numpy as np

# number of items in a pile
Items = 10

def play_random(state):
    pile = choice([i for i in range(3) if state[i] > 0])
    return randint(1, state[pile]), pile

def play_expert(state):
    xored = state[0] ^ state[1] ^ state[2]
    if xored == 0:
        return play_random(state)
    for pile in range(3):
        s = state[pile] ^ xored
        if s <= state[pile]:
            return state[pile] - s, pile

def play_qlearner(state):
    global qtable
    a = np.argmax(qtable[state[0], state[1], state[2]])  # exploitation
    move, pile = a % Items + 1, a // Items
    if move <= 0 or state[pile] < move:
        move, pile = play_random(state)  # exploration
    return move, pile

Then, we'll code up a simple turn-based game between any two players, each with its own algorithm.

In [2]:
from collections import defaultdict

Players = { 'Random':play_random, 'Expert':play_expert, 'Qlearner':play_qlearner }

def init_game():
    return [randint(1, Items), randint(1, Items), randint(1, Items)]

def play(n, a, b):
    wins = defaultdict(int)
    for _ in range(n):
        state, side = init_game(), False
        # play a game
        while True:
            player = Players[a] if not side else Players[b]
            move, pile = player(state)
            state[pile] -= move
            if state == [0, 0, 0]:
                break # someone wins
            side = not side
        wins[a if not side else b] += 1
    print(f"Play {n} games: {a:>10s} wins {wins[a]:4d}  {b:>10s} wins {wins[b]:4d}")
    return wins[a], wins[b]

To make the function `play_qlearner` works, we need to fill the so-called *Q table* used by the learner. What is that? This is a reinforcement learning (RL) technique called the *Q Learning*. The Q table is just a table filled with numbers called the *Q values*. These values are computed based on a mathematical equation called the *Bellman equation* which essentially follows a key principle called the *Markov property* that says *the future is independent of the past given the present - David Silver at DeepMind*. Poetic isn't it? 

Anyway, the Q table can be conceptually thought of as a two-dimensional array where the rows are indexed by all the states present in the system and the columns by the actions that could be taken from a state. In our case with the Nim game, a state is represented by a tuple of three integers of the remaining items in each pile while an action is a tuple of two integers of the number of items removed and the pile they are removed from i.e. the player's move.

Here is the Bellman equation in its formal mathematic form:

$$Q(s,a) = r + \gamma \max_{a'} Q(s',a')$$

And, here if we break it down into a version that iteratively updates the Q table:

$$Q(s,a) \leftarrow (1-\alpha)Q(s,a) + \alpha[r + \gamma \max_{a'} Q(s',a')]$$

$Q(s, a)$ in this equation is the current state Q value, $Q(s', a')$ the next state Q value. We call the $\alpha$ value the *learning rate* and $\gamma$ the *future discount* rate.

The following is the function that updates the Q values according to this equation.

In [3]:
qtable = None
Future_discount = 0.8

def update_qtable(reward, state, move, pile, future_best_q, learning_rate):
    a = pile * Items + move - 1
    current_q = qtable[state[0], state[1], state[2], a]
    qtable[state[0], state[1], state[2], a] = (1 - learning_rate) * current_q + learning_rate * (reward + Future_discount * future_best_q)

Our first RL training algorithm is a bit simplistic and doesn't do much, but it helps to visualize it in code to understand the basics of Q learning concepts.

In [4]:
Reward = 100
Learning_rate = 1.0
Epsilon_decay = 0.99

def train_qlearner(n):
    global qtable
    qtable = np.zeros((Items + 1, Items + 1, Items + 1, Items*3), dtype=np.float32)
    epsilon = 1.0 # Start with 100% exploration
    
    for _ in range(n):
        st1 = init_game()
        while True:
            if random() < epsilon:
                # exploration - make a random move
                move, pile = play_random(st1)
            else:
                # exploitation - pick the best action at the current state
                i = np.argmax(qtable[st1[0], st1[1], st1[2]])
                move, pile = i % Items + 1, i // Items                
                if move <= 0 or st1[pile] < move:
                    move, pile = play_random(st1)

            st2 = list(st1)
            st2[pile] -= min(move, st2[pile])
            
            if st2 == [0, 0, 0]:
                # game is over
                update_qtable(Reward, st1, move, pile, 0, Learning_rate)
                break
                
            update_qtable(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]), Learning_rate)

            st1 = st2
            
        epsilon = max(0.1, epsilon * Epsilon_decay)

In an RL training algorithm, we're always started off empty-handed. So to be able to keep populating the Q table, we need to first randomize our moves to mimic the so-called *exploration* activities. We use the *epsilon* value to control the rate in which the training explores as we want this rate to decay over time i.e. explore less and less, and do more and more of the opposite activity called the *exploitation* activities where we actually try to pick the best move at a time of the current state in the Q table. 

It doesn't look like we do much in this loop, doesn't it? But let's see how good the *Qlearner* player is after some randomness training. We'll train it for 1,000 rounds.

In [5]:
def eval(n):
    play(n, 'Qlearner', 'Random')
    play(n, 'Random', 'Qlearner')
    play(n, 'Qlearner', 'Expert')
    play(n, 'Expert', 'Qlearner')

train_qlearner(1000)
eval(1000)

Play 1000 games:   Qlearner wins  695      Random wins  305
Play 1000 games:     Random wins  291    Qlearner wins  709
Play 1000 games:   Qlearner wins    2      Expert wins  998
Play 1000 games:     Expert wins  997    Qlearner wins    3


That's actually pretty good for a random training act. Our *Qlearner* plays pretty well against the *Random* player no matter who starts first, but obviously is still horrible against the *Expert* player.

But now that we have some structure in place, can we train it to beat the *Expert*? What's the idea here? Yes, let's train our learner against the expert! To do that, we need to make a few more changes in the training loop.

In [6]:
Penalty = -100
Learning_rate = 0.8
Learning_rate_decay = 0.999999 # Alpha decay

def train_qlearner_tuned(n):
    global qtable
    qtable = np.zeros((Items + 1, Items + 1, Items + 1, Items*3), dtype=np.float32)
    epsilon = 1.0 # Start with 100% exploration
    learning_rate = Learning_rate
    
    for _ in range(n):
        st1 = init_game()
        turn = 1
        while True:
            if turn == 0:
                if random() < epsilon:
                    # exploration - make a random move
                    move, pile = play_random(st1)
                else:
                    # exploitation - pick the best action at the current state
                    i = np.argmax(qtable[st1[0], st1[1], st1[2]])
                    move, pile = i % Items + 1, i // Items                
                    if move <= 0 or st1[pile] < move:
                        move, pile = play_random(st1)
            else:
                move, pile = play_expert(st1)

            st2 = list(st1)
            st2[pile] -= min(move, st2[pile])
            
            if st2 == [0, 0, 0]:
                if turn == 0: # I won
                    update_qtable(Reward, st1, move, pile, 0, learning_rate)
                else: # I lost
                    update_qtable(Penalty, st1, move, pile, 0, learning_rate)
                break
                
            update_qtable(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]), learning_rate)

            st1 = st2
            turn = (turn + 1) % 2
            
        epsilon = max(0.1, epsilon * Epsilon_decay)
        learning_rate = max(0.1, Learning_rate * (Learning_rate_decay ** _))

The learner is now trained by playing against the *Expert* player via the `play_expert` function. If it wins the reward is deposited in the Q table, but when it loses a negative penalty is put in there. Notice that we're also started off with a 0.8 learning rate, why is that? The learning rate determines how much learning we take from the present vs. future state. In the previous simplistic training loop, we always set the learning rate to 1.0, which means we heavily lean on future learning by completely disregarding what happens at the present state. 

To make the learning more well-rounded and taking into account what the learner already learns from the present, a high but not 1.0 learning rate is what we will experiment with. This learning rate will also decay over time but very gradually, assuming we're going to need more rounds to train to successfully compete with the expert. Let's see how we do it this time. We'll now train it for 10,000 rounds.

In [7]:
train_qlearner_tuned(10000)
eval(1000)

Play 1000 games:   Qlearner wins  778      Random wins  222
Play 1000 games:     Random wins  221    Qlearner wins  779
Play 1000 games:   Qlearner wins  580      Expert wins  420
Play 1000 games:     Expert wins  960    Qlearner wins   40


Look! We beat the expert! The *Expert* player's one weakness is when the state of the piles is not what conveniently a strong position as determined by its mathematic method, which is much more likely to happen on the very first move of the game, since the Expert will be able to shape the state in the subsequent moves. In that one weak situation, the Expert simply defers to a random action. So, it seems that our *Qlearner* somehow discovers this weakness during the training and is able to successfully capitalize on it to beat the Expert when it has a chance to start a game! Evidently, it does so well that it manages to crush the Expert almost 60% of the time.

But, can we improve on this and make it even better?